In [124]:
import pandas as pd

In [125]:
# Read in the dataset
client = pd.read_csv('../data/CLIENT_191102.tsv',sep = '\t')
condition = pd.read_csv('../data/EE_UDES_191102.tsv',sep = '\t')
exit = pd.read_csv('../data/ENTRY_EXIT_191102.tsv',sep = '\t')

In [126]:
# Clean the dataset
client = client.drop(columns = ['EE Provider ID','Client ID'])
condition = condition.drop(columns = ['EE Provider ID','Entry Exit Provider Program Type Code','Client ID','Client Location(4378)','Zip Code (of Last Permanent Address, if known)(1932)','Relationship to Head of Household(4374)','Did you stay less than 7 nights?(5164)','Did you stay less than 90 days?(5163)','On the night before did you stay on the streets, ES or SH?(5165)','If yes for Domestic violence victim/survivor, when experience occurred(1917)'])
exit = exit.drop(columns = ['EE Provider ID','Client ID','Entry Exit Group Id','Entry Exit Household Id','Housing Move-in Date(5584)','Entry Exit Date Added','Entry Exit Date Updated','Entry Exit Type'])
exit.drop(exit.columns[2], axis=1, inplace=True)

Comparing the shape among three datasets, we found they have the same number of rows. Upon further investigation, the Client Unique ID are matched the same for each Entry Exit program unique ID. Since the client IDs are duplicated, we chose EE UID as primary key to combine the table.

In [127]:
# Combine the dataset
combined = pd.merge(client, condition, on = ['EE UID'])
combined = pd.merge(combined, exit, on =['EE UID']).drop(columns=['Client Unique ID_x','Client Unique ID_y'])
combined.columns

Index(['EE UID', 'Client Age at Entry', 'Client Age at Exit', 'Client Gender',
       'Client Primary Race', 'Client Ethnicity', 'Client Veteran Status',
       'Prior Living Situation(43)', 'Length of Stay in Previous Place(1934)',
       'Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)',
       'Total number of months homeless on the street, in ES or SH in the past three years(5168)',
       'Housing Status(2703)',
       'Does the client have a disabling condition?(1935)',
       'Covered by Health Insurance(4376)',
       'Domestic violence victim/survivor(341)', 'Date of Birth(893)',
       'Client Unique ID', 'Entry Date', 'Exit Date', 'Destination',
       'Reason for Leaving'],
      dtype='object')

In [128]:
# Wrangle the data: Remove the (HUD) from records tails
combined['Client Primary Race'] = combined['Client Primary Race'].str.slice(0,-5)
combined['Client Ethnicity'] = combined['Client Ethnicity'].str.slice(0,-5)
combined['Client Veteran Status'] = combined['Client Veteran Status'].str.slice(0,-5)
combined['Does the client have a disabling condition?(1935)'] = combined['Does the client have a disabling condition?(1935)'].str.slice(0,-5)
combined['Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)'] = combined['Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)'].str.slice(0,-5)
combined['Covered by Health Insurance(4376)'] = combined['Covered by Health Insurance(4376)'].str.slice(0,-5)
combined['Domestic violence victim/survivor(341)'] = combined['Domestic violence victim/survivor(341)'].str.slice(0,-5)
combined['Housing Status(2703)'] = combined['Housing Status(2703)'].str.slice(0,-5)
combined.head()

,EE UID,Client Age at Entry,Client Age at Exit,Client Gender,Client Primary Race,Client Ethnicity,Client Veteran Status,Prior Living Situation(43),Length of Stay in Previous Place(1934),"Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)",...,Housing Status(2703),Does the client have a disabling condition?(1935),Covered by Health Insurance(4376),Domestic violence victim/survivor(341),Date of Birth(893),Client Unique ID,Entry Date,Exit Date,Destination,Reason for Leaving
0,687901,60.0,61.0,Female,White,Non-Hispanic/Non-Latino,No,"Staying or living in a friend's room, apartmen...",One year or longer (HUD),Two times,...,Category 1 - Homeless,Yes,Yes,No,9/29/1954,pbkf09291954p610b236,8/15/2015,7/11/2016,"Rental by client, with other ongoing housing s...",Completed program
1,687902,48.0,48.0,Female,Black or African American,Non-Hispanic/Non-Latino,No,"Staying or living in a family member's room, a...","One month or more, but less than 90 days",Four or more times,...,Category 1 - Homeless,Yes,Yes,Yes,1/7/1967,kdaf01071967k400d635,8/15/2015,8/31/2015,Data not collected (HUD),Needs could not be met
2,687903,42.0,42.0,Female,Black or African American,Non-Hispanic/Non-Latino,No,"Staying or living in a friend's room, apartmen...","One month or more, but less than 90 days",Four or more times,...,Category 1 - Homeless,Yes,No,No,6/21/1973,smrf06211973s620m640,8/15/2015,9/19/2015,"Staying or living with friends, temporary tenu...",Other
3,687904,57.0,57.0,Male,White,Hispanic/Latino,No,"Staying or living in a friend's room, apartmen...",One year or longer (HUD),Four or more times,...,Category 1 - Homeless,Yes,No,No,7/25/1958,abrm07251958a416b600,8/15/2015,3/7/2016,Hospital or other residential non-psychiatric ...,Other
4,687905,51.0,51.0,Male,White,Non-Hispanic/Non-Latino,No,"Staying or living in a friend's room, apartmen...","One week or more, but less than one month",Four or more times,...,Category 1 - Homeless,Yes,No,No,1/25/1964,wbom01251964w450b620,8/15/2015,8/24/2015,"Staying or living with friends, temporary tenu...",Other


In [134]:
combined.columns = ['EEUID','Age_entry','Age_exit','Gender','Race','Ethnicity','Veteran','LivingSituation','LengthStayPrevious','TimesHomeless','LengthHomeless','HousingStatus','Disability','HealthInsurance','ViolanceVictim','DOB','ClientID','Entrydate','Exitdate','Destination','Reason']

In [137]:
combined.to_csv('../data/wrangle_data.csv')